In [1]:
%reload_ext autoreload
%autoreload 2

import os
from pathlib import Path

print(Path().cwd())
os.chdir(Path(os.getcwd()).parent)
print(Path().cwd())

/home/das/QuantUS_Projs/QuantUS-Plugins-CEUS/CLI-Demos
/home/das/QuantUS_Projs/QuantUS-Plugins-CEUS


## Select Contrast-Enhanced Ultrasound (CEUS) Cine and Parser

In [2]:
from src.image_loading.options import get_scan_loaders

print("Available scan loaders:", list(get_scan_loaders().keys()))

Available scan loaders: ['nifti']


In [3]:
scan_type = 'nifti'

scan_path = '/media/das/TOSHIBA EXT/ChinaData/3D-001_3rd/CEUS-25082 3rd.nii.gz'
scan_loader_kwargs = {
    'transpose': False,
    # 'resample_spacing': (0.7, 0.37, 0.52),
}

In [4]:
from src.entrypoints import scan_loading_step

image_data = scan_loading_step(scan_type, scan_path, **scan_loader_kwargs)

/home/das/QuantUS_Projs/QuantUS-Plugins-CEUS/src/image_loading/nifti/main.py:34: RuntimeWarning: divide by zero encountered in scalar divide
  frame_rate = 1.0 / header.get_zooms()[3] if len(header.get_zooms()) > 3 else None


In [5]:
image_data.pixel_data.shape, image_data.pixdim

((208, 172, 89, 281),
 (np.float32(0.99181175), np.float32(0.49220428), np.float32(0.5342902)))

## Load Segmentation

Assumes same segmentation for each frame

In [6]:
from src.seg_loading.options import get_seg_loaders

print("Available segmentation loaders:", list(get_seg_loaders().keys()))

Available segmentation loaders: ['nifti_voi', 'load_bolus_mask']


In [7]:
seg_type = 'nifti_voi'

seg_path = '/media/das/TOSHIBA EXT/ChinaData/3D-001_3rd/nifti_segmentation_QUANTUS/CEUS-25082_3rd_segmentation.nii.gz'
seg_loader_kwargs = {}

In [8]:
from src.entrypoints import seg_loading_step

seg_data = seg_loading_step(seg_type, image_data, seg_path, scan_path, **seg_loader_kwargs)

## CEUS Quantitative Temporal Curve Analysis

In [9]:
from src.time_series_analysis.options import get_analysis_types, get_required_kwargs

all_analysis_types, all_analysis_funcs = get_analysis_types()
print("Available analysis types:", list(all_analysis_types.keys()))
print("Available analysis functions:", list(all_analysis_funcs.keys()))

Available analysis types: ['curves', 'curves_paramap']
Available analysis functions: ['pyradiomics', 'tic']


In [10]:
analysis_type = 'curves_paramap'
analysis_funcs = ['tic']

# Find all required kwargs for the analysis functions
analysis_funcs = analysis_funcs if len(analysis_funcs) else list(all_analysis_funcs[analysis_type].keys())
required_kwargs = get_required_kwargs(analysis_type, analysis_funcs)
print("Required kwargs for current analysis:", required_kwargs)

Required kwargs for current analysis: ['sag_vox_ovrlp', 'ax_vox_len', 'sag_vox_len', 'cor_vox_len', 'cor_vox_ovrlp', 'ax_vox_ovrlp']


In [12]:
analysis_kwargs = {
    'ax_vox_ovrlp': 50,  # %
    'sag_vox_ovrlp': 50,  # %
    'cor_vox_ovrlp': 50,  # %
    'ax_vox_len': 10.0,  # mm
    'sag_vox_len': 5.0,  # mm
    'cor_vox_len': 5.0,  # mm
    'curves_output_path': '', # don't export the curves we generate
}

In [13]:
from src.entrypoints import analysis_step

analysis_obj = analysis_step(analysis_type, image_data, seg_data, analysis_funcs, **analysis_kwargs)

Computing curves: 100%|██████████| 281/281 [01:00<00:00,  4.64it/s]


## Curve Quantification

In [14]:
from src.curve_quantification.options import get_quantification_funcs

quantification_funcs = get_quantification_funcs()
print("Available quantification functions:", quantification_funcs.keys())

Available quantification functions: dict_keys(['auc_no_fit', 'cmus_firstorder', 'dte', 'first_order', 'lognormal_fit', 'wash_rates'])


In [15]:
function_names = ['lognormal_fit'] # Empty list will use all functions
output_path = 'test_quants.csv'
curve_quantifications_kwargs = {
    'curves_to_fit': ['TIC'],
}

In [16]:
from src.entrypoints import curve_quantification_step

curve_quant = curve_quantification_step(analysis_obj, function_names, output_path, **curve_quantifications_kwargs)

## Parametric Map Visualizations

In [17]:
from src.visualizations.options import get_visualization_types

types, funcs = get_visualization_types()
print("Available visualization types:", list(types.keys()))
print("Available visualization functions:", list(funcs.keys()))

Available visualization types: ['paramap']
Available visualization functions: []


In [19]:
vis_type = 'paramap'
params = [] # Empty list will use all parameters 
vis_funcs = []
vis_kwargs = {
    'paramap_folder_path': 'paramaps_LOGNORMAL',
    'hide_all_visualizations': False,  # Set to True to hide all visualizations
}

In [21]:
from src.entrypoints import visualization_step

vis_obj = visualization_step(curve_quant, vis_type, params, vis_funcs, **vis_kwargs)